In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout


In [2]:
# Load IMDB dataset with top 10,000 most frequent words
vocab_size = 10000
maxlen = 200

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)


x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

print("Training samples:", len(x_train))
print("Test samples:", len(x_test))


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Training samples: 25000
Test samples: 25000


In [3]:
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=maxlen))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [4]:
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=5,
                    validation_data=(x_test, y_test),
                    verbose=2)


Epoch 1/5
391/391 - 263s - 671ms/step - accuracy: 0.7795 - loss: 0.4709 - val_accuracy: 0.8224 - val_loss: 0.3999
Epoch 2/5
391/391 - 262s - 671ms/step - accuracy: 0.8522 - loss: 0.3515 - val_accuracy: 0.8465 - val_loss: 0.3591
Epoch 3/5
391/391 - 263s - 673ms/step - accuracy: 0.8796 - loss: 0.2981 - val_accuracy: 0.8345 - val_loss: 0.3785
Epoch 4/5
391/391 - 262s - 670ms/step - accuracy: 0.9091 - loss: 0.2352 - val_accuracy: 0.8586 - val_loss: 0.3464
Epoch 5/5
391/391 - 321s - 820ms/step - accuracy: 0.9180 - loss: 0.2106 - val_accuracy: 0.8183 - val_loss: 0.5240


In [5]:
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print(f"✅ Test Accuracy: {acc:.4f}")


✅ Test Accuracy: 0.8183


In [6]:

word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

def decode_review(text_ids):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in text_ids])

sample = x_test[0]
print("Review:", decode_review(x_test[0]))
print("True Label:", y_test[0])

prediction = model.predict(sample.reshape(1, -1))[0][0]
print("Predicted Sentiment:", "Positive" if prediction > 0.5 else "Negative")


1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Review: ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? please give this one a miss br br ? ? and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite ? so all you madison fans give this a miss
True Label: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step
Predicted Sentiment: Negative
